## Импорты

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import itertools as it
import time
import warnings
import sys

from scipy import stats

warnings.filterwarnings('ignore', '.*p-value.*')
sys.setrecursionlimit(11000)

## Генерация датафрейма

### 1. Генерация идентификаторов

In [3]:
def generate_user_id(rng, symbols_list):
  """Функция возвращает список сгенерированных идентификаторов"""
  # Генерируем список идентификаторов посимвольно выбирая случайный символ из исходной строки
  user_id_list = [''.join(rng.choice(symbols_list, 15)) for _ in range(0, 10000)]

  assert len(user_id_list) == 10000

  return user_id_list

### 2. Генерация номеров заказа

In [4]:
def generate_order_numbers(rng):
  """Функция возвращает список сгенерированных номеров заказа"""
  order_number_list = rng.integers(low=1, high=11, size=10000)

  assert len(order_number_list) == 10000
  assert min(order_number_list) == 1 and max(order_number_list) == 10

  return order_number_list

### 3. Генерация времени доставки

In [5]:
def generate_click2delivery(rng):
  """Функция возвращает сгенерированный список времени доставки"""
  click2delivery_list = rng.normal(loc=1440, scale=200, size=10000)

  assert len(click2delivery_list) == 10000
  # Тест Шапиро-Уилка чтобы проверить является ли распределение нормальным и numpy нас не обманывает при использовании rng.normal()
  assert stats.shapiro(click2delivery_list).pvalue >= 0.05

  return click2delivery_list

### 4. Генерация стоимости заказа

Для генерации требуемого распределения numpy оказалось недостаточно, так как в формуле в numpy отсутствует возможность передать смещение. В связи с этим применил Scipy.

In [6]:
def generate_order_items_sum():
  """Функция возвращает сгенерированный список времени доставки"""
  # Чтобы получить lambda 1 передаём scale = 1, так как scale = 1 / lambda
  order_items_sum_list = stats.expon.rvs(loc=1, scale=1, size=10000, random_state=42)

  assert len(order_items_sum_list) == 10000

  return order_items_sum_list

### 5. Генерация дня жизни покупателя

In [7]:
def generate_retention(rng):
  """Функция возвращает сгенерированный список дня жизни покупателя в который совершён заказ"""
  retention_list = rng.choice(a=[1, 2, 3, 4, 5], p=[0.35, 0.25, 0.2, 0.15, 0.05], size=10000)

  assert len(retention_list) == 10000
  assert min(retention_list) == 1 and max(retention_list) == 5

  return retention_list

### 6. Создание датафрейма и удаление дублирующихся идентификаторов

In [8]:
def generate_df():
  """
    Функция возвращает датафрейм из сгенерированных в соответствии с заданием значений
  """
  # seed указан для того чтобы результат был воспроизводим и сгенерированные значения не отличались от запуска к запуску
  rng = np.random.default_rng(seed=42)

  user_id_list = generate_user_id(rng, list("1234567890abcdefghijk"))
  order_number_list = generate_order_numbers(rng)
  click2delivery_list = generate_click2delivery(rng)
  order_items_sum_list = generate_order_items_sum()
  retention_list = generate_retention(rng)

  d = {'user_id': user_id_list, 'order_number': order_number_list, 'click2delivery': click2delivery_list, 'order_items_sum': order_items_sum_list, 'retention': retention_list}

  df = pd.DataFrame(data=d)

  # Проверка на дубликаты в столбце с идентификаторами
  if bool(df.duplicated(subset="user_id").sum()):
    df['user_id'].drop_duplicates(inplace=True)

  return df

In [9]:
df = generate_df()

display(df.head(15))

,user_id,order_number,click2delivery,order_items_sum,retention
0,2gd00i2e52bkfff,2,1506.586727,1.469268,1
1,ga3h0a84jgd9hb0,10,1428.108708,4.010121,1
2,052bi2ih6d4fe82,1,1472.059736,2.316746,2
3,k0iegf580a1b4fe,3,1369.932979,1.912943,3
4,jf8k97j820g403e,6,1277.290555,1.169625,2
5,075bej04hde37gh,10,1630.726904,1.169596,4
6,0gh9i76ed3h5g1g,8,1396.556608,1.059839,1
7,ggd0e6gb0ab1363,9,1681.327068,3.011231,2
8,0ed0hb2gcdbb2bg,5,1430.831454,1.919082,2
9,7c180k569kh15h2,9,1436.006023,2.231250,1


## Задачи

### 1. Среднее по группе 

In [10]:
# Для начала группирую значения по номеру заказа, считаю среднее и выбираю нужный столбец
click2delivery_by_number = df.groupby('order_number').mean()['click2delivery'].rename('mean_click2delivery_by_order_number')

# Джойню по индексу и перезаписываю результат в df
df = df.merge(click2delivery_by_number, how='left', left_on='order_number', right_on=click2delivery_by_number.index)

display(df.head())

,user_id,order_number,click2delivery,order_items_sum,retention,mean_click2delivery_by_order_number
0,2gd00i2e52bkfff,2,1506.586727,1.469268,1,1431.651325
1,ga3h0a84jgd9hb0,10,1428.108708,4.010121,1,1436.300484
2,052bi2ih6d4fe82,1,1472.059736,2.316746,2,1445.288830
3,k0iegf580a1b4fe,3,1369.932979,1.912943,3,1437.720544
4,jf8k97j820g403e,6,1277.290555,1.169625,2,1439.008563


### 2. Столбец с последовательностью

Задача похожа на формирование последовательности Фибоначчи с добавлением одного действия (умножения суммы на 0.5). 

Наиболее простое решение "в лоб" здесь использовать рекурсию предварительно вырубив стандартное Python ограничение на её глубину.

In [11]:
def generate_element(n, seq, end):
  """
    Функция возвращает следующий элемент последовательности

    Arguments:
          n (int): Позиция следующего элемента в списке
          seq (list): Исходная последовательность
          end (int): Точка остановки рекурсии

    Returns:
          sequence (pd.Series): Итоговая последовательность длиной end
  """
  if n < end:
    # Обработка ситуации когда в исходной последовательности один элемент
    if n == 1:
      seq.append(seq[n - 1] * 0.5)
    else:
      seq.append((seq[n - 1] + seq[n - 2]) * 0.5)

    return generate_element(len(seq), seq, end)
  else:
    return seq

seq = [0.1]

generate_element(len(seq), seq, 10000)

df['sequence'] = seq

display(df.head())

,user_id,order_number,click2delivery,order_items_sum,retention,mean_click2delivery_by_order_number,sequence
0,2gd00i2e52bkfff,2,1506.586727,1.469268,1,1431.651325,0.10000
1,ga3h0a84jgd9hb0,10,1428.108708,4.010121,1,1436.300484,0.05000
2,052bi2ih6d4fe82,1,1472.059736,2.316746,2,1445.288830,0.07500
3,k0iegf580a1b4fe,3,1369.932979,1.912943,3,1437.720544,0.06250
4,jf8k97j820g403e,6,1277.290555,1.169625,2,1439.008563,0.06875


### 3. Обработка user_id

In [12]:
def id_processing(val):
  """
    Функция обрабатывает идентификаторы в соответствии с заданием

    Arguments:
          val (str): Один идентификатор из столбца

    Returns:
          res (str): Обработанный идентификатор 
  """
  # Для начала получаем все буквы из идентификатора
  letters = ''.join([s for s in val if not s.isdigit()])
  # Тем же самым методом без отрицания получаем все числа и собираем в одно число
  numbers = ''.join([s for s in val if s.isdigit()])

  # Преобразуем получившееся число в integer и возводим в квадрат, затем преобразуем обратно в строку для конкатенации с строкой letters
  if bool(len(numbers)):
    square_str = str(int(numbers) ** 2)
  # Обработка случая когда в строке нет ни одного числа
  else:
    square_str = ''

  res = letters + square_str

  assert len(res) == len(letters) + len(square_str)
  
  return res

### 4. Добавление нового столбца в таблицу

Первой реакцией на задачу было использовать apply, но задумавшись и почитав документацию понял что она под капотом по сути является оптимизированным для датафрейма циклом for

Не совсем понятно подходит ли np.vectorise по условию задачи, в документации numpy в заметках к функции указано что реализация тоже по сути представляет собой цикл for. 
Однако, использует преимущества numpy по части broadcasting что сильно ускоряет обработку

In [13]:
id_processing_vec = np.vectorize(id_processing)
arr = df['user_id'].to_numpy()

df['new_id'] = id_processing_vec(arr)

display(df.head())

,user_id,order_number,click2delivery,order_items_sum,retention,mean_click2delivery_by_order_number,sequence,new_id
0,2gd00i2e52bkfff,2,1506.586727,1.469268,1,1431.651325,0.10000,gdiebkfff40100863504
1,ga3h0a84jgd9hb0,10,1428.108708,4.010121,1,1436.300484,0.05000,gahajgdhb95166080100
2,052bi2ih6d4fe82,1,1472.059736,2.316746,2,1445.288830,0.07500,biihdfe27316114096324
3,k0iegf580a1b4fe,3,1369.932979,1.912943,3,1437.720544,0.06250,kiegfabfe3365624196
4,jf8k97j820g403e,6,1277.290555,1.169625,2,1439.008563,0.06875,jfkjge806081476043082409


### 4.1 Замеры скорости исполнения кода

In [14]:
start_time = time.time()

id_processing_vec = np.vectorize(id_processing)

id_processing_vec(df['user_id'].to_numpy())

print(f'Скорость np.vectorize: {time.time() - start_time}')

start_time = time.time()

df['user_id'].apply(id_processing)

print(f'Скорость pandas apply: {time.time() - start_time}')

Скорость np.vectorize: 0.05399680137634277
Скорость pandas apply: 0.057178497314453125


Откровенно говоря разницы особо нет, apply отрабатывает чаще всего даже немного побыстрее

Предполагаю что причина в том что работа с несколькими действиями производимыми со строками (часть из которых изменение типов данных туда-сюда) - не самый оптимальный сценарий для того чтобы применять эту функцию к вектору

### 5. describe()?

In [15]:
def print_stat_values(series):
  """Функция принимает на вход series датафрейма и выводит на экран требуемые по заданию стат. показатели  """
  print(f'Столбец: {series.name}')
  # mode() немного непредсказуемо работает с float значениями без округления
  print(f'Мода: {series.round(0).mode()[0]}')
  print(f'Медиана: {series.median()}')
  print(f'Среднее арифметическое: {series.mean()}')
  print(f'Дисперсия: {series.var()}')
  print(f'Стандартное отклонение: {series.std()}')
  print('\n')

print_stat_values(df['click2delivery'])
print_stat_values(df['order_items_sum'])
print_stat_values(df['retention'])

Столбец: click2delivery
Мода: 1482.0
Медиана: 1436.0834059498409
Среднее арифметическое: 1437.6757949733153
Дисперсия: 40956.15491550288
Стандартное отклонение: 202.3762706334487


Столбец: order_items_sum
Мода: 1.0
Медиана: 1.6783149586461987
Среднее арифметическое: 1.9774989546902346
Дисперсия: 0.9494645328118946
Стандартное отклонение: 0.9744047068912868


Столбец: retention
Мода: 1
Медиана: 2.0
Среднее арифметическое: 2.3054
Дисперсия: 1.5222830683068307
Стандартное отклонение: 1.233808359635657




Не уверен что смысл задания был в том чтобы я вспомнил какие методы в Pandas выводят нужные стат. значения, но писать собственный велосипед для вывода этих значений ещё более странно.

### 6. Гистограммы распределения значений

In [16]:
fig = px.histogram(df, x="click2delivery")
fig.show()

fig = px.histogram(df, x="order_items_sum")
fig.show()

fig = px.histogram(df, x="retention")
fig.show()

### 7. Зависимость времени доставки от номера

In [17]:
fig = px.scatter(df, y="click2delivery", x="order_number")
fig.show()

Scatterplot в принципе традиционно используется когда нужно показать зависимость одной переменной от другой.

В качестве альтернативы можно попробовать barplot или lineplot, но с ними возникнут серьёзные проблемы:

1) barplot по умолчанию аггрегирует y-значение, вместо того чтобы показывать конкретные точки. В некоторых кейсах этого может быть достаточно, например когда мы показываем изменения метрики во времени. Но в нашем кейсе это лишит нас информации о распределении y.

2) lineplot имеет те же самые проблемы что и barplot (не покажет распределение значений, так как будет аггрегировать значения), и, кроме того, неадекватно будет вести себя с дискретным x